# Bedrock의 Claude 모델을 활용하여 OCR 텍스트와 원본 이미지를 분석하고, 번역가들이 사용할 최종 파일을 생성하기
(이미지와 OCR 텍스트를 Claude가 분석하여 의미적 그룹으로 분류)

## 1. 환경 셋업

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install Pillow openpyxl pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import sys
import boto3
import json
from typing import Dict, List, Optional
from datetime import datetime
import os
import base64
from botocore.config import Config


# utils 모듈 임포트
from utils import (
    encode_image, 
    read_html_content, 
    get_image_format,
    create_translation_workflow,
    add_python_path,
    check_file_paths,
    list_ocr_results
)

# Python 경로 설정
module_path = ".."
add_python_path(module_path)


python path: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery is added
sys.path:  ['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/joohyery/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery']


In [ ]:
# AWS Bedrock 클라이언트 설정

region = "us-west-2"
nova_pro_model_id = "us.amazon.nova-pro-v1:0"
claude3_7_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

config = Config(
    read_timeout=300,  # 이미지 + 텍스트 멀티모달 처리를 위해 5분으로 설정
)

client = boto3.client(service_name="bedrock-runtime", region_name=region, config=config)


# 2. OCR 결과 파일 경로 로딩 하기

In [5]:
# 현재 작업 디렉토리 확인
current_folder = os.getcwd()
print(f"현재 작업 디렉토리: {current_folder}")

# OCR 결과 파일 목록 확인
list_ocr_results()

# 기본 파일 경로
image_path = "samples/sample1.jpg"
html_path = "ocr-results/try_upstage_document_ai.html"

# 파일 존재 여부 확인
check_file_paths(image_path, html_path)

현재 작업 디렉토리: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate
OCR 결과 파일들:
  - try_upstage_document_ai.json
  - try_upstage_document_ai.html
이미지 파일: samples/sample1.jpg (존재)
HTML 파일: ocr-results/try_upstage_document_ai.html (존재)


(True, True)

In [ ]:
def llm_converse(image_base64, html_content, model_id):
    """Claude 모델에 Converse API를 사용하여 이미지와 텍스트를 함께 전송하여 분석 요청"""
    
    # 이미지 형식 자동 감지
    image_format = get_image_format(image_path)
    print(image_format)
    
    # 시스템 프롬프트
    system_prompt = """
    당신은 OCR 텍스트와 원본 이미지를 분석하여 번역가가 사용할 수 있도록 텍스트를 의미적 그룹으로 분류하는 전문가입니다.
    
    주어진 이미지와 OCR 텍스트를 분석하여 다음과 같이 처리해주세요:
    
    1. 이미지의 레이아웃과 텍스트의 배치를 파악
    2. 의미적으로 연관된 텍스트들을 그룹화
    
    결과는 JSON 형태로 반환해주세요:
    {
        "groups": [
            {
                "category": "그룹 카테고리명",
                "description": "그룹 설명",
                "texts": ["텍스트1", "텍스트2", ...]
            }
        ]
    }
    """
    
    user_prompt = f"""
    다음은 OCR로 추출된 HTML 입니다:
    
    {html_content}
    
    이 텍스트와 함께 제공된 원본 이미지를 분석하여, 번역가가 효율적으로 작업할 수 있도록 의미적으로 연관된 텍스트들을 그룹화해주세요.
    """
    
    try:
        # Converse API 호출
        response = client.converse(
            modelId=claude3_7_model_id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "image": {
                                "format": image_format,
                                "source": {
                                    "bytes": base64.b64decode(image_base64)
                                }
                            }
                        },
                        {
                            "text": user_prompt
                        }
                    ]
                }
            ],
            system=[
                {
                    "text": system_prompt
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.1
            }
        )
        
        # 응답에서 텍스트 추출
        response_text = response['output']['message']['content'][0]['text']
        return response_text
        
    except Exception as e:
        raise Exception(f"Claude 모델 호출 중 오류 발생: {str(e)}")

In [ ]:
def main_process_converse(image_path, html_path):
    """전체 프로세스를 실행하는 메인 함수 (Converse API 사용)"""
    
    print("=== OCR 텍스트와 이미지 분석 시작 (Converse API) ===")
    
    try:
        # 1. 이미지 인코딩
        print("1. 이미지를 인코딩 중...")
        image_base64 = encode_image(image_path)
        print("   이미지 인코딩 완료")
        
        # 2. HTML 내용 읽기
        print("2. OCR 결과를 읽는 중...")
        html_content = read_html_content(html_path)
        print(f"   HTML 내용 길이: {len(html_content)} 문자")
        
        # 3. 모델로 전사 작업
        print("3. LLM 모델로 텍스트 그룹 분석 중...")
        
        claude_response = llm_converse(image_base64, html_content, claude3_7_model_id)
        
        print("   Claude 분석 완료")
        
        # 4. JSON 파싱 
        print("4. 결과를 파싱 중...")
        try:
            json_text = claude_response.strip()
            
            # 마크다운 JSON 코드 블록 찾기
            if "```json" in claude_response:
                json_start = claude_response.find("```")
                json_end = claude_response.find("```", json_start)
                
                if json_end != -1:
                    json_text = claude_response[json_start:json_end].strip()
                else:
                    json_text = claude_response[json_start:].strip()
            
            parsed_result = json.loads(json_text)
            grouped_texts = parsed_result.get('groups', [])
            
        except json.JSONDecodeError as e:
            print(f"   JSON 파싱 실패: {str(e)}")
            print("   텍스트를 단순 그룹으로 처리합니다.")
            grouped_texts = [{"category": "General", "texts": [claude_response]}]
        
        print(f"   총 {len(grouped_texts)}개의 텍스트 그룹 생성")
        
        # 5. 번역 문서 생성
        print("5. 번역 문서 생성 중...")
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        final_file_path = create_translation_workflow(
            grouped_texts=grouped_texts,
            image_name=image_name,
            source_lang="Korean",
            target_lang="English"
        )
        
        print("\n=== 전체 프로세스 완료 ===")
        print(f"최종 파일: {final_file_path}")
        
        return final_file_path, grouped_texts
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None, None


In [13]:
image_exists, html_exists = check_file_paths(image_path, html_path)

if image_exists and html_exists:
    print("\n" + "="*50)
    print("프로세스를 시작합니다... (Converse API)")
    print("="*50)
    
    # 일반 모드 실행
    print("\n>>> 일반 모드로 실행 중...")
    final_file, groups = main_process_converse(image_path, html_path)
    
    if final_file:
        print(f"\n✅ 성공적으로 완료되었습니다!")
        print(f"📁 번역 문서 위치: {final_file}")
        print(f"📊 텍스트 그룹 수: {len(groups) if groups else 0}")
    else:
        print("❌ 프로세스 실행 중 오류가 발생했습니다.")
else:
    print("❌ 필요한 파일이 없어서 프로세스를 실행할 수 없습니다.")

이미지 파일: samples/sample1.jpg (존재)
HTML 파일: ocr-results/try_upstage_document_ai.html (존재)

프로세스를 시작합니다... (Converse API)

>>> 일반 모드로 실행 중...
=== OCR 텍스트와 이미지 분석 시작 (Converse API) ===
1. 이미지를 인코딩 중...
원본 이미지 크기: 1000x17717
이미지 크기가 너무 큽니다. 리사이징 중... (최대: 8000x8000)
리사이징된 이미지 크기: 451x8000
   이미지 인코딩 완료
2. OCR 결과를 읽는 중...
   HTML 내용 길이: 9147 문자
3. LLM 모델로 텍스트 그룹 분석 중 (Converse API)...
   Claude 분석 완료
4. 결과를 파싱 중...
   JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
   텍스트를 단순 그룹으로 처리합니다.
   총 1개의 텍스트 그룹 생성
5. 번역 문서 생성 중...
번역 문서를 포맷팅 중...
번역 문서를 저장 중...
'/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate/final_results' 폴더를 생성했습니다.
번역 문서가 저장되었습니다: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate/final_results/translation_document_sample1_20250805_092454.xlsx
총 1개의 텍스트 그룹이 포함되어 있습니다.

=== 번역 문서 생성 완료 ===
파일 경로: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate/final_results/translation_document_samp